In [1]:
from tqdm import tqdm
from datetime import date
import bs4, requests, json

In [2]:
url = r'https://news-eleven.com/news'

# get original page
soup = bs4.BeautifulSoup(requests.get(url).content, 'lxml')

# get main menu links
main_menu_list = soup.select('#block-system-main-menu')
print('# main-menu:', len(main_menu_list))

# get all the links from main menu
main_menu_links_list = [atag['href'] for atag in main_menu_list[0].ul.find_all('a')]
print('# main menu links:', len(main_menu_links_list))

# main-menu: 1
# main menu links: 31


In [3]:
# fetch the article links from each main menu links
art_url_list = []
for mml in tqdm(main_menu_links_list):
    current_page = bs4.BeautifulSoup(requests.get(mml).content, 'lxml')
    
    # each of these pages in main menu contains `front page`, `most read` and `recent news`
    # get front-page links
    front_page_divs = current_page.select('.frontpage-title')
    front_page_links = [t.a['href'] for t in front_page_divs]
#     print('There are {} links.'.format(len(front_page_links)))
    art_url_list.extend(front_page_links)

    # get most read links
    most_read_divs = current_page.select('.most-read-title')
    most_read_links = [t.a['href'] for t in most_read_divs]
#     print('There are {} links.'.format(len(most_read_links)))
    art_url_list.extend(most_read_links)

    # get most recent links
    recent_news_divs = current_page.select('.recent-news-title')
    recent_news_links = [t.a['href'] for t in recent_news_divs]
#     print('There are {} links.'.format(len(recent_news_links)))
    art_url_list.extend(recent_news_links)

print('# articles:', len(art_url_list))

100%|██████████| 31/31 [00:28<00:00,  1.09it/s]

# articles: 820


In [6]:
art_url_list_unq = list(set(art_url_list))
print(len(art_url_list), len(art_url_list_unq))

820 320


In [4]:
N = 50 # we are only taking 50

article_list = []
for art_url in tqdm(art_url_list_unq[:N]):
    article = bs4.BeautifulSoup(requests.get(art_url).content, 'lxml')
    art_title = article.select('.news-detail-title')[0].text
    art_body = article.select('.node-content')
    article_list.append({
        'url': art_url,
        'title': art_title,
        'body': '\n'.join(para.text for para in art_body[0].find_all('p'))
    })

NameError: name 'art_url_list_unq' is not defined

In [ ]:
with open('data/eleven_{}.json'.format(date.today()), 'w', encoding='utf-8') as f:
    json.dump(article_list, f, ensure_ascii=False, indent=4)